In [1]:
from backtesting import Strategy
from backtesting.lib import crossover
from backtesting import Backtest
import pandas as pd
from backtesting.test import SMA
import pandas as pd
from bokeh.io import export_png
from backtesting.lib import SignalStrategy, TrailingStrategy
from backtesting import Position

/Users/fahad/anaconda3/lib/python3.7/site-packages/backtesting/_plotting.py:34: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `bokeh.io.reset_output()`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [2]:
AMZN_predictions = pd.read_csv("/Users/fahad/project_repo/models/final_models/Financial_Analysis/predictions/AMZN_predictions_SVM.csv", index_col=0)


In [3]:
AMZN_predictions.head()

,Date,Predictions
0,2019-06-28,0
1,2019-05-31,0
2,2019-04-30,1
3,2019-03-29,0
4,2019-02-28,1


In [4]:
AMZN_predictions.Date = pd.to_datetime(AMZN_predictions.Date, format="%Y-%m-%d")

In [5]:
AMZN_predictions['Predictions'] = AMZN_predictions['Predictions'].shift(-1)


In [6]:
AMZN_predictions = AMZN_predictions.dropna()


In [7]:
AMZN_predictions = AMZN_predictions[::-1]

In [8]:
AMZN_predictions.head(10)

,Date,Predictions
111,2010-03-31,1.0
110,2010-04-30,0.0
109,2010-05-28,1.0
108,2010-06-30,1.0
107,2010-07-30,1.0
106,2010-08-31,1.0
105,2010-09-30,1.0
104,2010-10-29,1.0
103,2010-11-30,0.0
102,2010-12-31,0.0


In [9]:
AMZN_monthly = pd.read_csv("/Users/fahad/project_repo/data/external/AMZN_monthly.csv", index_col =0)

In [10]:
AMZN_monthly.columns = ['Open','High','Low','Close','Volume']

In [11]:
AMZN_predictions = AMZN_predictions.set_index("Date")

In [12]:
AMZN_monthly.head()

,Open,High,Low,Close,Volume
2019-07-26,1922.98,2035.80,1906.63,1943.05,61308702
2019-06-28,1760.01,1935.20,1672.00,1893.63,74746589
2019-05-31,1933.09,1964.40,1772.70,1775.07,98214360
2019-04-30,1800.11,1956.34,1798.73,1926.52,81293632
2019-03-29,1655.13,1823.75,1586.57,1780.75,100832035


In [13]:
AMZN_final_df = pd.merge(AMZN_predictions, AMZN_monthly, right_index=True, left_index=True)

In [14]:
AMZN_final_df.index = pd.to_datetime(AMZN_final_df.index, format ="%Y-%m-%d")

In [15]:
equity_dict = dict()

In [16]:
class Binary_Classification_1d(TrailingStrategy):

    
    def init(self):
        
        super().init()
        self.data.Predictions
        self.set_trailing_sl(4)
        
    def next(self):
        super().init()
        equity_dict[self.data.index[-1]] = TrailingStrategy.equity.__get__(self)
        
        position = Strategy.position.__get__(self)
        
        if self.data.Predictions[-1] == 1.0 and (position.is_long!=True):
#             print("\n")
#             print (self.data.Prediction[-1])
            
            self.buy()

        # Else, if sma1 crosses below sma2, sell it
        elif self.data.Predictions[-1] == 0.0 and (position.is_short!=True) :
#             print("\n")
#             print (self.data.Prediction[-1])
            
            self.sell()
            
 

In [17]:
bt = Backtest(AMZN_final_df, Binary_Classification_1d, cash=10000, commission=.01)
bt.run()



Start                          2010-03-31 00:00:00
End                            2019-06-28 00:00:00
Duration                        3376 days 00:00:00
Exposure [%]                                98.282
Equity Final [$]                            636506
Equity Peak [$]                             636506
Return [%]                                 6265.06
Buy & Hold Return [%]                      1294.73
Max. Drawdown [%]                         -23.6044
Avg. Drawdown [%]                         -9.06101
Max. Drawdown Duration           514 days 00:00:00
Avg. Drawdown Duration           157 days 00:00:00
# Trades                                        47
Win Rate [%]                               74.4681
Best Trade [%]                             48.4535
Worst Trade [%]                             -11.53
Avg. Trade [%]                             9.72129
Max. Trade Duration              274 days 00:00:00
Avg. Trade Duration               71 days 00:00:00
Expectancy [%]                 

In [18]:
bt.plot(superimpose=False, plot_volume=False)

In [19]:
df = pd.DataFrame.from_dict([equity_dict])

In [20]:
df = df.T

In [21]:
df.columns = ['Equity']

In [22]:
export = df.to_csv("/Users/fahad/project_repo/data/final/Equity/AMZN_SVM_Equity.csv")

In [23]:
df

,Equity
2010-04-30,10000.000000
2010-05-28,10748.203100
2010-06-30,9334.613999
2010-07-30,10071.916935
2010-08-31,10664.834939
2010-09-30,13418.400829
2010-10-29,14116.403725
2010-11-30,14985.276362
2010-12-31,15083.907114
2011-01-31,15960.248959
